In [16]:
from ollama_client import OllamaClient
from pydantic import BaseModel, Field
from tools import tavily_search
import datetime
import json
from openai_client import OpenAIClient as OpenAIClient
from deep_research_tools import deep_tavily_search
print(datetime.datetime.now().strftime("%d %b %Y"))

# loadenv
import dotenv
dotenv.load_dotenv()

provider = 'ollama'
if provider == "openai":
    Client  = OpenAIClient
    model1 = 'gpt-5-nano'
    model2 = 'gpt-5-mini'

else:
    Client = OllamaClient
    model1 = "qwen3:4b"
    model2 = "qwen3:8b"



25 Aug 2025


### Reformulate User Question

In [17]:
task = """you are an expert on simplifying user question for optimal google search. 
1. Identify 3 different  formulations of what the user is asking
2. keep the reformulations short and concise
3. the 3 formulations must be varied and must be useful and varied variations of the user query but must be strongly related to the user query
4. No need to write complete sentences but iinclude all important aspects"""

re_phrase_agent = Client(role = task, model_name=model1)

class RePhraseResult(BaseModel):
    formulation_1: str
    formulation_2: str
    formulation_3: str


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

{'text': '{\n  "formulation_1": "improve python code performance",\n  "formulation_2": "optimize python code speed",\n  "formulation_3": "python performance tuning"\n}\n\n',
 'raw': {'model': 'qwen3:4b',
  'created_at': '2025-08-24T20:52:04.286280725Z',
  'message': {'role': 'assistant',
   'content': '{\n  "formulation_1": "improve python code performance",\n  "formulation_2": "optimize python code speed",\n  "formulation_3": "python performance tuning"\n}\n\n'},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 8157306281,
  'load_duration': 4209239415,
  'prompt_eval_count': 448,
  'prompt_eval_duration': 1074315380,
  'eval_count': 43,
  'eval_duration': 2864002034},
 'conversation_history': [{'role': 'user',
   'content': 'Hey Man. i love coding. \n                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?'},
  {'role': 'assistant',
   'content': '{\n  "formulation_1": "improve python code performan

In [3]:
print(re_phrase_agent.system_instructions)

You are a helpful Agent among a group of agents trying to solve a problem. Each agent is tasked with a part or the entire problem.
You will be given your task. You will have access to all the relevant information and tools. You can also see the work already done by other agents. Use that information if required.


## Instructions
1. View the context, the task executed, the results, the tool call results of the other agents.
2. Reason and execute your task.
3. You have access to multiple tools as listed. You can only call tools which are relevant to your task. Other agents might have executed other tools which you dont have access to.
4. Always output strictly JSON. Always.
5. your task will be enclosed in <YOUR TASK></YOUR TASK> tags. This is your task. Only execute this task.
6. The work done by other agents will be enclosed in <Agent: Agent Name></Agent: Agent Name> tags. There may be multiple of these.


Following is the relevant information from other agents (if any):



<YOUR TASK

### Web Search Agent

In [ ]:
task = """You are an expert on simplifying user question for optimal google search and then invoking search tool. 
1. Identify 3 different formulations of what the user is asking
2. keep the reformulations short and concise. 
3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the user question 3 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. Rephrase the user question in 3 ways and pass each of them to the tool
"""

re_phrase_agent = Client(role = task, model_name=model1, agent_name="SearchAgent")

class AnswerBasedOnContext(BaseModel):
    answer: str

search_queries = re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: What is the latest version of gpt out there?""", tools=[tavily_search])

search_queries

{'text': '',
 'raw': {'id': 'chatcmpl-C8BWFwbG9yZGVxxY8m1BDixeDRuO3',
  'object': 'chat.completion',
  'created': 1756066939,
  'model': 'gpt-5-nano-2025-08-07',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': None,
     'tool_calls': [{'id': 'call_c0ku2NoKG6KBLt38jPEtv1bd',
       'type': 'function',
       'function': {'name': 'tavily_search',
        'arguments': '{"query1":"What is the latest GPT version available today?", "query2":"What is the most recent GPT model release?", "query3":"What is the newest version of GPT in the market right now?"}'}}],
     'refusal': None,
     'annotations': []},
    'finish_reason': 'tool_calls'}],
  'usage': {'prompt_tokens': 821,
   'completion_tokens': 576,
   'total_tokens': 1397,
   'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
   'completion_tokens_details': {'reasoning_tokens': 512,
    'audio_tokens': 0,
    'accepted_prediction_tokens': 0,
    'rejected_prediction_tokens': 0}},
  'servi

In [4]:
import json
print(search_queries['tool_results']["tavily_search"])
print(len(search_queries['tool_results']["tavily_search"]))

<search_metadata>
Queries: ['What is the latest GPT version available today?', 'What is the most recent GPT model release?', 'What is the newest version of GPT in the market right now?']
Total Queries: 3
Max Results per Query: 15
Search Mode: Advanced
Raw Content: False
</search_metadata>

<query_section>
Query: What is the latest GPT version available today?
</query_section>

<chunk 1>
Query: What is the latest GPT version available today?
Title: What's new in Azure OpenAI in Azure AI Foundry Models?
URL: https://learn.microsoft.com/en-us/azure/ai-foundry/openai/whats-new
Score: 0.82186085
Published: N/A
Content: ### Evaluations guide ### Latest GPT-4o model available in the early access playground (preview) On August 6, 2024, OpenAI announced the latest version of their flagship GPT-4o model version `2024-08-06`. GPT-4o `2024-08-06` has all the capabilities of the previous version as well as: `2024-08-06` `2024-08-06` Azure customers can test out GPT-4o `2024-08-06` today in the new 

## Deep Research 1


#### Deep Research Clarification Asker

In [18]:
task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
2. keep the questions short and concise.
3. The questions should clarify what the user wants to research / learn / understand / plan out.
4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

For example, if the user asks: 
Example 1:
User Question: Help me prepare an itinerary for my trip to Japan next month.
Clarifying questions: 
1. What specific cities in Japan are you planning to visit?
2. How many days do you have for your trip?
3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
4. What is your budget for the trip?

Example 2:
User Question: Which city are you moving to?
Clarifying Questions:
1. Which city are you moving to?
2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
3. What’s your estimated move-in date?
4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
5. What is your budget for the move and for monthly living expenses?


Example 3:
User Question:
I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

Clarifying Questions:
1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
"""

clarifying_agent = Client(role = task, model_name=model1, agent_name="ClarifyingAgent")

class ClarifyingQuestions(BaseModel):
    question_1: str
    question_2: str
    question_3: str


#clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
clarifying_questions = clarifying_agent.invoke(query="""Help me understand where to buy good coffee beans""", 
                                               json_schema=ClarifyingQuestions)



clarifying_questions

{'text': '{\n  "question_1": "What is your budget range for coffee beans per pound?",\n  "question_2": "Do you prefer single-origin or blended coffee beans?",\n  "question_3": "Are you looking for beans that are roasted recently or are you okay with beans that have been roasted a few weeks ago?"\n}\n',
 'raw': {'model': 'qwen3:4b',
  'created_at': '2025-08-24T20:58:20.224181594Z',
  'message': {'role': 'assistant',
   'content': '{\n  "question_1": "What is your budget range for coffee beans per pound?",\n  "question_2": "Do you prefer single-origin or blended coffee beans?",\n  "question_3": "Are you looking for beans that are roasted recently or are you okay with beans that have been roasted a few weeks ago?"\n}\n'},
  'done_reason': 'stop',
  'done': True,
  'total_duration': 10844438464,
  'load_duration': 4202159855,
  'prompt_eval_count': 781,
  'prompt_eval_duration': 1735077112,
  'eval_count': 69,
  'eval_duration': 4892940373},
 'conversation_history': [{'role': 'user',
   'c

{'question_1': 'What is your budget range for coffee beans per pound?',
 'question_2': 'Do you prefer single-origin or blended coffee beans?',
 'question_3': 'Are you looking for beans that are roasted recently or are you okay with beans that have been roasted a few weeks ago?'}

In [ ]:
def deep_research_clarification_agent(deep_research_request, model=model1):
    task = """You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

    1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
    2. keep the questions short and concise.
    3. The questions should clarify what the user wants to research / learn / understand / plan out.
    4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

    For example, if the user asks: 
    Example 1:
    User Question: Help me prepare an itinerary for my trip to Japan next month.
    Clarifying questions: 
    1. What specific cities in Japan are you planning to visit?
    2. How many days do you have for your trip?
    3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
    4. What is your budget for the trip?

    Example 2:
    User Question: Which city are you moving to?
    Clarifying Questions:
    1. Which city are you moving to?
    2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
    3. What’s your estimated move-in date?
    4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
    5. What is your budget for the move and for monthly living expenses?


    Example 3:
    User Question:
    I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

    Clarifying Questions:
    1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
    2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
    3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
    4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)
    """

    clarifying_agent = Client(role = task, model_name=model, agent_name="ClarifyingAgent")

    class ClarifyingQuestions(BaseModel):
        question_1: str
        question_2: str
        question_3: str


    #clarifying_questions = re_phrase_agent.invoke(query="""Help me plan a birthday party""", json_schema=ClarifyingQuestions)
    clarifying_questions = clarifying_agent.invoke(query="""Help me understand where to buy good coffee beans""", 
                                                json_schema=ClarifyingQuestions)

    

    return json.loads(clarifying_questions['text'])

In [11]:
user_answers = """\n\nuser_answers: """ + """
1. Online Purchases
2. Light Roast and whole beans
3. Freshly roasted
"""

In [12]:
from IPython.display import display, Markdown
Markdown(clarifying_agent.get_agent_context())

<Agent: ClarifyingAgent>
            ### Task to be done by the agent:
            You are an expert on asking clarifying Questions which will let you perform deep research on the user question. 

1. Identify 3 different questions to ask the user the answers to which will help you answer the user question.
2. keep the questions short and concise.
3. The questions should clarify what the user wants to research / learn / understand / plan out.
4. Once the user answers the questions, use their responses to guide your research and provide more tailored information.

For example, if the user asks: 
Example 1:
User Question: Help me prepare an itinerary for my trip to Japan next month.
Clarifying questions: 
1. What specific cities in Japan are you planning to visit?
2. How many days do you have for your trip?
3. What type of activities are you interested in (e.g., cultural, outdoor, culinary)?
4. What is your budget for the trip?

Example 2:
User Question: Which city are you moving to?
Clarifying Questions:
1. Which city are you moving to?
2. What’s your reason for the move (e.g., job, studies, lifestyle change)?
3. What’s your estimated move-in date?
4. Do you need help finding housing, setting up utilities, understanding the local job market, or anything else?
5. What is your budget for the move and for monthly living expenses?


Example 3:
User Question:
I want to understand Autoencoders. Help me understand the key architectures, starting from basics to good models

Clarifying Questions:
1. What is your primary purpose for learning about autoencoders? (e.g., anomaly detection, dimensionality reduction, data generation, etc.)
2. What level of detail are you looking for in the architecture explanations? Do you want theoretical grounding, mathematical formulation, and implementation-level details (e.g., PyTorch or TensorFlow)?
3. Are you interested only in standard autoencoders or also in variants like variational autoencoders (VAEs), denoising autoencoders, sparse autoencoders, etc.?
4. What is your level of current understanding? (e.g., beginner, intermediate, advanced)


            ### User Input to the agent: Help me understand where to buy good coffee beans
            ### Agent's Response: {"question_1":"What is your location and do you prefer online purchases, local roasters, or both?","question_2":"What are your bean preferences (origin vs. blends, roast level, whole bean vs ground, grind size if you have a grinder)?","question_3":"Any constraints or priorities (budget, freshness, shipping speed, sustainability certifications like Organic/Direct Trade)?"}
            
            </Agent: ClarifyingAgent>

### Deep Research Search Agent

In [ ]:
task = """You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. 
Observe the shared conversation between user and Agent and generate 6 questions which, when searched can answer the user's intent.
1. Identify 6 questions based on the conversation on which web search can be done.
2. keep the search questions short and concise. 
3. Invoke the deep_tavily_search if it will help provide an updated and more accurate answer to the question.
4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.
5. If the question needs to refer to documentation, do a web search and answer
6. If the user asks for recent events or asks to refer to the web, then use this tool
7. Rephrase the user question in 6 ways and pass each of them to the tool
"""

deep_search_question_formulator_1 = Client(role = task, model_name=model1, agent_name="DeepSearchAgent1")

class DeepSearchAgent1(BaseModel):
    question1: str
    question2: str
    question3: str
    question4: str
    question5: str
    question6: str

search_queries = deep_search_question_formulator_1.invoke(query=f"""{clarifying_agent.get_agent_context()}"""+user_answers,
                                                           tools=[deep_tavily_search])

search_queries

{'text': '',
 'raw': {'id': 'chatcmpl-C8BbeiX71muGbTETU0ej9Ww7x4Ajb',
  'object': 'chat.completion',
  'created': 1756067274,
  'model': 'gpt-5-nano-2025-08-07',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': None,
     'tool_calls': [{'id': 'call_DlbUDYDWx1ioGPHw5J3jfjXe',
       'type': 'function',
       'function': {'name': 'deep_tavily_search',
        'arguments': '{"query1":"Where can I buy freshly roasted light roast coffee beans online?","query2":"Which online shops sell light roast whole-bean coffee that’s freshly roasted?","query3":"Where to buy fresh light roast coffee beans online (whole bean)?","query4":"Best online retailers for freshly roasted light roast whole-bean coffee.","query5":"Online sources for fresh, light-roast, whole-bean coffee?","query6":"Where can I order freshly roasted light roast whole bean coffee online with quick shipping?"}'}}],
     'refusal': None,
     'annotations': []},
    'finish_reason': 'tool_calls'}],
  'us

In [30]:
json.loads(search_queries['text'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
re_phrase_agent.get_agent_context()

"<Agent: DeepSearchAgent1>\n            Task to be done by the agent:\nThis is the You are an expert on observing Agent-User conversations and understanding user intent for making search tool calls. \nObserve the shared conversation between user and Agent and generate 5 questions which, when searched can answer the user's intent.\n1. Identify 6 questions based on the conversation on which web search can be done.\n2. keep the search questions short and concise. \n3. Invoke the tavily_search if it will help provide an updated and more accurate answer to the question.\n4. If invoking the tool, extract from the agent-user question 6 different ways of formulating the same question and pass to the search tool.\n5. If the question needs to refer to documentation, do a web search and answer\n6. If the user asks for recent events or asks to refer to the web, then use this tool\n7. Rephrase the user question in 6 ways and pass each of them to the tool\n\n\n            ### User Input to the agent

### Generic Tool calling function

In [ ]:
def call_tools():
    

### Agent that summarizes the Answer

In [6]:
task = """You are an Expert on Answering a User Question based on provided context. 
Study the context deeply and answer the question asked by the user. Use a table in the end to summarize the answer"""
model1="qwen3:1.7b"
model2="llama3.2-vision:11b"
model3="gemma3:4b"


re_phrase_agent = OllamaClient(role = task, model_name=model2)

class AnswerBasedOnContext(BaseModel):
    answer: str
    


re_phrase_agent.invoke(query="""Hey Man. i love coding. 
                       I wanted to ask you several questions one of which is: How can I improve the performance of my Python code?""", json_schema=RePhraseResult)

HTTPError: 404 Client Error: Not Found for url: http://localhost:11434/api/chat